In [292]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import Huber
from tensorflow.data import Dataset
from tensorflow import expand_dims

In [298]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/
! kaggle datasets download -d selfishgene/historical-hourly-weather-data

kaggle.json
historical-hourly-weather-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [297]:
! unzip -o /content/historical-hourly-weather-data.zip

Archive:  /content/historical-hourly-weather-data.zip
  inflating: city_attributes.csv     
  inflating: humidity.csv            
  inflating: pressure.csv            
  inflating: temperature.csv         
  inflating: weather_description.csv  
  inflating: wind_direction.csv      
  inflating: wind_speed.csv          


In [ ]:
path = '/content/wind_speed.csv'
df = pd.read_csv(path)
df

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,San Antonio,Dallas,Houston,Kansas City,Minneapolis,Saint Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN
1,2012-10-01 13:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,3.0,1.0,0.0,3.0,4.0,0.0,4.0,4.0,3.0,0.0,3.0,4.0,3.0,0.0,3.0,4.0,7.0,4.0,3.0,1.0,0.0,8.0,2.0,2.0,2.0
2,2012-10-01 14:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,3.0,1.0,0.0,3.0,4.0,0.0,4.0,4.0,3.0,0.0,3.0,4.0,3.0,0.0,3.0,4.0,7.0,4.0,3.0,3.0,0.0,8.0,2.0,2.0,2.0
3,2012-10-01 15:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,3.0,1.0,0.0,3.0,4.0,0.0,4.0,4.0,3.0,0.0,3.0,4.0,3.0,0.0,3.0,3.0,7.0,4.0,3.0,3.0,0.0,8.0,2.0,2.0,2.0
4,2012-10-01 16:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,3.0,1.0,0.0,3.0,3.0,0.0,4.0,4.0,3.0,0.0,3.0,4.0,3.0,0.0,3.0,3.0,7.0,4.0,3.0,3.0,0.0,8.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45248,2017-11-29 20:00:00,NaN,2.0,NaN,4.0,1.0,2.0,1.0,3.0,1.0,3.0,2.0,3.0,2.0,3.0,8.0,4.0,2.0,2.0,3.0,1.0,1.0,3.0,1.0,NaN,3.0,4.0,5.0,NaN,10.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
45249,2017-11-29 21:00:00,NaN,1.0,NaN,5.0,1.0,1.0,1.0,1.0,1.0,6.0,2.0,3.0,2.0,2.0,7.0,4.0,4.0,4.0,3.0,1.0,2.0,4.0,1.0,NaN,2.0,4.0,5.0,NaN,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
45250,2017-11-29 22:00:00,NaN,1.0,NaN,5.0,1.0,2.0,1.0,2.0,1.0,3.0,2.0,3.0,3.0,2.0,7.0,5.0,2.0,3.0,3.0,1.0,3.0,3.0,1.0,NaN,2.0,2.0,3.0,NaN,7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
45251,2017-11-29 23:00:00,NaN,2.0,NaN,3.0,2.0,2.0,0.0,3.0,1.0,1.0,3.0,2.0,2.0,2.0,5.0,2.0,2.0,2.0,1.0,1.0,1.0,3.0,0.0,NaN,2.0,4.0,3.0,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
TARGET = 'Saint Louis'
df[df[TARGET].isnull()]

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,San Antonio,Dallas,Houston,Kansas City,Minneapolis,Saint Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN


In [ ]:
targetDf = df.dropna()[['datetime', TARGET]]
targetDf

,datetime,Saint Louis
1,2012-10-01 13:00:00,4.0
2,2012-10-01 14:00:00,4.0
3,2012-10-01 15:00:00,4.0
4,2012-10-01 16:00:00,3.0
5,2012-10-01 17:00:00,3.0
...,...,...
44456,2017-10-27 20:00:00,7.0
44457,2017-10-27 21:00:00,7.0
44458,2017-10-27 22:00:00,8.0
44459,2017-10-27 23:00:00,6.0


In [ ]:
def create_windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = expand_dims(series, axis=-1)
    ds = Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
size = targetDf.shape[0]
test_ratio = 0.2
test_size = int(size * test_ratio)
test_start = size - test_size
train = targetDf.iloc[:test_start]
test = targetDf.iloc[test_start:]

In [ ]:
train

,datetime,Saint Louis
1,2012-10-01 13:00:00,4.0
2,2012-10-01 14:00:00,4.0
3,2012-10-01 15:00:00,4.0
4,2012-10-01 16:00:00,3.0
5,2012-10-01 17:00:00,3.0
...,...,...
35555,2016-10-21 23:00:00,2.0
35556,2016-10-22 00:00:00,2.0
35557,2016-10-22 01:00:00,2.0
35558,2016-10-22 02:00:00,1.0


In [ ]:
test

,datetime,Saint Louis
35560,2016-10-22 04:00:00,1.0
35561,2016-10-22 05:00:00,1.0
35562,2016-10-22 06:00:00,1.0
35563,2016-10-22 07:00:00,1.0
35564,2016-10-22 08:00:00,1.0
...,...,...
44456,2017-10-27 20:00:00,7.0
44457,2017-10-27 21:00:00,7.0
44458,2017-10-27 22:00:00,8.0
44459,2017-10-27 23:00:00,6.0


In [ ]:
BATCH_SIZE = 64
SHUFFLE = 640
WINDOW = 48
train_ds = create_windowed_dataset(train[TARGET].values, WINDOW, BATCH_SIZE, SHUFFLE)
test_ds = create_windowed_dataset(test[TARGET].values, WINDOW, BATCH_SIZE, SHUFFLE)

In [ ]:
def defineModel():
  layers = [
            LSTM(512, return_sequences=True),
            LSTM(256, return_sequences=True),
            LSTM(128),
            Dropout(0.2),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dropout(0.2),
            Dense(16, activation='relu'),
            Dense(1)
]
  model = Sequential(layers)
  model.compile(loss = Huber(), optimizer = Nadam(learning_rate = 0.0005), metrics = ['mae'])
  return model
model = defineModel()

In [ ]:
CHECKPOINT = './checkpoints/checkpoint.ckpt'
earlyStopping = EarlyStopping(monitor = 'loss', 
                              mode = 'min', 
                              patience = 5
)
checkpoint = ModelCheckpoint(filepath = CHECKPOINT,
                             monitor = 'val_mae', 
                             save_weights_only=True, 
                             mode = 'min', 
                             save_best_only = True
)
callbacks = [
        earlyStopping,
        checkpoint
]

In [ ]:
model.fit(
    train_ds,
    verbose = 1,
    epochs = 200,
    steps_per_epoch = 8,
    callbacks = callbacks,
    validation_data = test_ds
)

Epoch 1/200
8/8 [==============================] - 103s 13s/step - loss: 1.7753 - mae: 2.2146 - val_loss: 1.0133 - val_mae: 1.4417
Epoch 2/200
8/8 [==============================] - 61s 8s/step - loss: 1.1021 - mae: 1.5335 - val_loss: 1.1530 - val_mae: 1.5763
Epoch 3/200
8/8 [==============================] - 60s 8s/step - loss: 1.2455 - mae: 1.6829 - val_loss: 0.9272 - val_mae: 1.3555
Epoch 4/200
8/8 [==============================] - 59s 8s/step - loss: 1.1503 - mae: 1.5791 - val_loss: 0.7917 - val_mae: 1.2043
Epoch 5/200
8/8 [==============================] - 60s 8s/step - loss: 1.4108 - mae: 1.8469 - val_loss: 0.8576 - val_mae: 1.2820
Epoch 6/200
8/8 [==============================] - 60s 8s/step - loss: 1.1796 - mae: 1.6104 - val_loss: 0.6953 - val_mae: 1.0887
Epoch 7/200
8/8 [==============================] - 61s 8s/step - loss: 1.2001 - mae: 1.6423 - val_loss: 0.6948 - val_mae: 1.0920


In [ ]:
scale = max(targetDf[TARGET]) - min(targetDf[TARGET])

In [ ]:
final_model = defineModel()
final_model.load_weights(CHECKPOINT)
loss, mae = final_model.evaluate(test_ds)

139/139 [==============================] - 56s 385ms/step - loss: 0.6953 - mae: 1.0887


In [ ]:
print(f'data scale: {scale}')
print(f'mae: {mae}')
print(f'mae ratio vs data scale: {mae / scale * 100}%')

data scale: 21.0
mae: 1.0887442827224731
mae ratio vs data scale: 5.18449658439273%
